# Prediction of CWE

In [1]:
from sklearn.utils import shuffle
import vulns_common

vulns_common.download_nvd_vulns_json()
nvd_vulns = vulns_common.load_nvd_vulns_json('data/nvdcve-1.0*.json')
print('Vulnerability descriptions: ' + str(len(nvd_vulns)))
cpe_names = vulns_common.compile_cpe_names(nvd_vulns)
print('Vulnerable product names:'+str(len(cpe_names)))
data = shuffle(nvd_vulns)#, n_samples=2000)
print('data: ' + str(len(data)))

Vulnerability descriptions: 32963
Vulnerable product names:9176
data: 32963


In [2]:
data[0]

['CVE-2018-5518',
 'On F5 BIG-IP 13.0.0-13.1.0.5 or 12.0.0-12.1.3.3, malicious root users with access to a VCMP guest can cause a disruption of service on adjacent VCMP guests running on the same host. Exploiting this vulnerability causes the vCMPd process on the adjacent VCMP guest to restart and produce a core file. This issue is only exploitable on a VCMP guest which is operating in "host-only" or "bridged" mode. VCMP guests which are "isolated" are not impacted by this issue and do not provide mechanism to exploit the vulnerability. Guests which are deployed in "Appliance Mode" may be impacted however the exploit is not possible from an Appliance Mode guest. To exploit this vulnerability root access on a guest system deployed as "host-only" or "bridged" mode is required.',
 'NVD-CWE-noinfo',
 'AV:A/AC:M/Au:S/C:N/I:N/A:P',
 2.3,
 'CVSS:3.0/AV:A/AC:H/PR:H/UI:N/S:C/C:N/I:N/A:H',
 5.4,
 ['f5',
  'big-ip_local_traffic_manager',
  'f5',
  'big-ip_application_acceleration_manager',
  'f5'

In [3]:
deprecated_cwes = [1,3,10,60,63,68,70,71,92,100,101,132,139,169,217,218,225,247,249,292,373,418,423,443,445,458,503,504,505,513,516,517,518,533,534,542,545,592,596,630,631,632,633,634,679,769]
cwe_counts = dict()

def update_cwe_counts(data_):
    counts_ = dict()
    for d in data_:
        cwe = d[2]
        try:
            cwe_id = int(cwe[4:len(cwe)])
        except:
            cwe_id = 0
        if cwe_id in deprecated_cwes:
            if cwe == 'CWE-534':#This entry has been deprecated because its abstraction was too low-level. See CWE-532.
                cwe = 'CWE-532'
            if cwe == 'CWE-769':#This entry has been deprecated because it was a duplicate of CWE-774.
                cwe = 'CWE-774'
        if cwe in counts_.keys():
            counts_[cwe] = counts_[cwe] + 1
        else:
            counts_[cwe] = 1
    return counts_
cwe_counts = update_cwe_counts(data)
cwe_counts

{'NVD-CWE-noinfo': 1986,
 'CWE-79': 3849,
 'CWE-275': 139,
 'CWE-94': 296,
 'CWE-200': 2254,
 'CWE-416': 929,
 'CWE-287': 646,
 'CWE-125': 1377,
 'CWE-119': 2680,
 'CWE-74': 257,
 'CWE-326': 81,
 'CWE-20': 2834,
 'CWE-476': 466,
 'CWE-772': 117,
 'CWE-352': 882,
 '': 1550,
 'CWE-426': 153,
 'CWE-285': 102,
 'CWE-862': 79,
 'CWE-281': 8,
 'CWE-502': 187,
 'CWE-284': 819,
 'CWE-19': 62,
 'CWE-320': 47,
 'CWE-190': 787,
 'CWE-264': 586,
 'CWE-400': 422,
 'CWE-184': 5,
 'CWE-399': 128,
 'CWE-732': 293,
 'CWE-89': 943,
 'CWE-617': 32,
 'CWE-77': 365,
 'NVD-CWE-Other': 50,
 'CWE-191': 30,
 'CWE-78': 493,
 'CWE-522': 146,
 'CWE-312': 61,
 'CWE-319': 64,
 'CWE-269': 261,
 'CWE-787': 558,
 'CWE-22': 957,
 'CWE-306': 80,
 'CWE-295': 200,
 'CWE-434': 311,
 'CWE-798': 261,
 'CWE-665': 30,
 'CWE-668': 42,
 'CWE-310': 235,
 'CWE-693': 43,
 'CWE-362': 141,
 'CWE-611': 324,
 'CWE-415': 108,
 'CWE-311': 53,
 'CWE-909': 19,
 'CWE-209': 20,
 'CWE-601': 183,
 'CWE-427': 56,
 'CWE-918': 154,
 'CWE-704': 21

In [4]:
len(cwe_counts)

160

In [5]:
vulns_common.download_cwe_csv()
f = vulns_common.CweFinder()
for i, (k, v) in enumerate(cwe_counts.items()):
    cwe = k
    if v > 10 or not cwe.startswith('CWE-'):
        continue
    cwe = cwe[4:len(cwe)]
    cwe = f.find_root_cwe(cwe)#finds the root of a requested cwe
    if len(cwe) > 0:
        cwe = 'CWE-' + cwe
        for j, d in enumerate(data):
            if d[2] == k:
                data[j][2] = cwe
cwe_counts = update_cwe_counts(data)

In [6]:
len(cwe_counts)

118

In [7]:
cwe_counts

{'NVD-CWE-noinfo': 1986,
 'CWE-79': 3849,
 'CWE-275': 139,
 'CWE-94': 296,
 'CWE-200': 2254,
 'CWE-416': 929,
 'CWE-287': 646,
 'CWE-125': 1377,
 'CWE-119': 2680,
 'CWE-74': 257,
 'CWE-326': 81,
 'CWE-20': 2834,
 'CWE-476': 466,
 'CWE-772': 117,
 'CWE-352': 882,
 '': 1550,
 'CWE-426': 153,
 'CWE-285': 102,
 'CWE-862': 79,
 'CWE-693': 76,
 'CWE-502': 187,
 'CWE-284': 819,
 'CWE-19': 62,
 'CWE-320': 47,
 'CWE-190': 787,
 'CWE-264': 586,
 'CWE-400': 422,
 'CWE-399': 128,
 'CWE-732': 293,
 'CWE-89': 943,
 'CWE-617': 32,
 'CWE-77': 365,
 'NVD-CWE-Other': 50,
 'CWE-191': 30,
 'CWE-78': 493,
 'CWE-522': 146,
 'CWE-312': 61,
 'CWE-319': 64,
 'CWE-269': 261,
 'CWE-787': 558,
 'CWE-22': 957,
 'CWE-306': 80,
 'CWE-295': 200,
 'CWE-434': 311,
 'CWE-798': 261,
 'CWE-665': 30,
 'CWE-668': 42,
 'CWE-310': 235,
 'CWE-362': 141,
 'CWE-611': 324,
 'CWE-415': 108,
 'CWE-311': 53,
 'CWE-909': 19,
 'CWE-209': 20,
 'CWE-601': 183,
 'CWE-427': 56,
 'CWE-918': 154,
 'CWE-704': 213,
 'CWE-307': 28,
 'CWE-327':

In [8]:
cwes_to_remove = []
for i, (k, v) in enumerate(cwe_counts.items()):
    if v < 10:# or not k.startswith('CWE-'):
        cwes_to_remove.append(k)

for i, d in enumerate(data):
    if d[2] in cwes_to_remove:
        data[i][2] = 'Unknown'

cwe_counts = update_cwe_counts(data)

In [9]:
cwe_counts

{'NVD-CWE-noinfo': 1986,
 'CWE-79': 3849,
 'CWE-275': 139,
 'CWE-94': 296,
 'CWE-200': 2254,
 'CWE-416': 929,
 'CWE-287': 646,
 'CWE-125': 1377,
 'CWE-119': 2680,
 'CWE-74': 257,
 'CWE-326': 81,
 'CWE-20': 2834,
 'CWE-476': 466,
 'CWE-772': 117,
 'CWE-352': 882,
 '': 1550,
 'CWE-426': 153,
 'CWE-285': 102,
 'CWE-862': 79,
 'CWE-693': 76,
 'CWE-502': 187,
 'CWE-284': 819,
 'CWE-19': 62,
 'CWE-320': 47,
 'CWE-190': 787,
 'CWE-264': 586,
 'CWE-400': 422,
 'CWE-399': 128,
 'CWE-732': 293,
 'CWE-89': 943,
 'CWE-617': 32,
 'CWE-77': 365,
 'NVD-CWE-Other': 50,
 'CWE-191': 30,
 'CWE-78': 493,
 'CWE-522': 146,
 'CWE-312': 61,
 'CWE-319': 64,
 'CWE-269': 261,
 'CWE-787': 558,
 'CWE-22': 957,
 'CWE-306': 80,
 'CWE-295': 200,
 'CWE-434': 311,
 'CWE-798': 261,
 'CWE-665': 30,
 'CWE-668': 42,
 'CWE-310': 235,
 'CWE-362': 141,
 'CWE-611': 324,
 'CWE-415': 108,
 'CWE-311': 53,
 'CWE-909': 19,
 'CWE-209': 20,
 'CWE-601': 183,
 'CWE-427': 56,
 'CWE-918': 154,
 'CWE-704': 213,
 'CWE-307': 28,
 'CWE-327':

In [10]:
len(cwe_counts)

112

In [11]:
## Handling the class imbalance

In [12]:
from sklearn.utils import resample
import pandas as pd

def sample_cwe(data_, max_samples):
    df = pd.DataFrame(data_)
    df = df.drop([0,3,4,5,6,7], axis=1)
    df = df.drop(df[df[2] == 'Unknown'].index)
    df = df.drop(df[df[2] == 'NVD-CWE-noinfo'].index)
    df = df.drop(df[df[2] == ''].index)
    df = df.rename(index=str, columns={1: 'text', 2: 'label'})

    cwe_counts = update_cwe_counts(data_)
    cwe_texts = pd.DataFrame()

    #print('Max samples: '+ str(max_samples))
    for i, (k, v) in enumerate(cwe_counts.items()):
        if k == '' or k == 'Unknown' or k == 'NVD-CWE-noinfo':
            continue
        if v > max_samples:
            resampled = resample(df[df['label'] == k], replace=False, n_samples=max_samples)
            #print('Downsampling: '+k)
        else:
            resampled = df[df['label'] == k]
        cwe_texts = pd.concat([cwe_texts, resampled])

    return shuffle(cwe_texts)

cwe_texts = sample_cwe(data, 1000000)
print(len(cwe_texts))

29394


In [13]:
cwe_texts.head(10)

,text,label
16152,A malicious authenticated user may be able to ...,CWE-284
25350,A vulnerability in the web-based management in...,CWE-79
29022,GNOME Evolution through 3.28.2 is prone to Ope...,CWE-347
9027,This vulnerability allows remote attackers to ...,CWE-416
4834,Dell EMC Enterprise Copy Data Management (eCDM...,CWE-295
1990,In firmware version MS_2.6.9900 of Columbia We...,CWE-79
3824,Reflected XSS on Micro Focus Enterprise Develo...,CWE-79
24330,A Cross-site Scripting (XSS) vulnerability exi...,CWE-79
1811,A shell escape vulnerability in /webconsole/Co...,CWE-78
20342,An XSS issue was discovered in the checklist p...,CWE-79


In [14]:
## classification with classifiers suggested https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html

In [15]:
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction import text
import numpy as np
import time

class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        self.stemmer = SnowballStemmer("english")
        analyzer = super(CountVectorizer, self).build_analyzer()
        return lambda doc: (analyzer(' '.join([self.stemmer.stem(word) for word in doc.split(' ')])))

class StemmedTfidfVectorizer(TfidfVectorizer):
    def build_analyzer(self):
        self.stemmer = SnowballStemmer("english")
        analyzer = super(TfidfVectorizer, self).build_analyzer()
        return lambda doc: (analyzer(' '.join([self.stemmer.stem(word) for word in doc.split(' ')])))

class LemmaCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        self.stemmer = WordNetLemmatizer()
        analyzer = super(CountVectorizer, self).build_analyzer()
        return lambda doc: (analyzer(' '.join([self.stemmer.lemmatize(word) for word in doc.split(' ')])))    

class LemmaTfidfVectorizer(TfidfVectorizer):
    def build_analyzer(self):
        self.stemmer = WordNetLemmatizer()
        analyzer = super(TfidfVectorizer, self).build_analyzer()
        return lambda doc: (analyzer(' '.join([self.stemmer.lemmatize(word) for word in doc.split(' ')])))    

unwanted_words = ['issue','defect','bug','fault','flaw','mistake','error','version','system','because','before','disputed']
stop_words = text.ENGLISH_STOP_WORDS#.union(cpe_names)
stop_words = stop_words.union(unwanted_words)

vectorizer = TfidfVectorizer(stop_words=stop_words,
                             lowercase=True,
                             ngram_range=(1, 2), 
                             min_df=1,
                             token_pattern=r'(?u)\b\w*[a-zA-Z]{3,}\w*\b')

classifiers = [
               #MultinomialNB(),
               #SGDClassifier(tol=1e-3, shuffle=True),
               LinearSVC(),#dual=False, max_iter=2000, class_weight='balanced'
               #KNeighborsClassifier()
              ]

nfold = 10

for classifier in classifiers:
    t = time.time()
    print(str(classifier)+'\n')
    pipe = Pipeline([('vect', vectorizer), ('cls', classifier)])
    scores = []
    scores = cross_val_score(pipe, cwe_texts['text'], cwe_texts['label'], cv=nfold, scoring="f1_micro")

    print(str(nfold)+'-fold cross validated f1-score for CWE:' + str(round(np.mean(scores),3)))
    elapsed_time = time.time() - t
    print('Time taken: ' + str(round(elapsed_time, 1)) + 's\n')

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

10-fold cross validated f1-score for CWE:0.843
Time taken: 231.1s



In [16]:
# See the amount of features
pipe = Pipeline([('vect', vectorizer), ('cls', LinearSVC())])
pipe.fit(cwe_texts['text'], cwe_texts['label'])
vectors = vectorizer.transform(cwe_texts['text'])
print("Vectors shape:" + str(vectors.shape))
#vectorizer.vocabulary_

Vectors shape:(29394, 211444)


In [17]:
# Test prediction of CWE on a security related bug report

In [18]:
import pandas as pd
import random

reports = pd.read_excel('data/sec_issues_edit.xlsx')
reports = reports.loc[(reports['security']==1)]
reports['security'] = reports['security'].values.astype(bool)
reports['report'] = reports['report'].values.astype(str)
i = random.randint(0, len(reports)-1)
random_report = reports.iloc[i]['report']

In [19]:
print(random_report)
predicted = pipe.predict([random_report])
cwe_id = predicted[0]
cwe_id

Login page password-guessing attack A common threat web developers face is a password-guessing attack known as a brute force attack. A brute-force attack is an attempt to discover a password by systematically trying every possible combination of letters, numbers, and symbols until you discover the one correct combination that works. 

This login page doesn't have any protection against password-guessing attacks (brute force attacks). It's recommended to implement some type of account lockout after a defined number of incorrect password attempts. Consult Web references for more information about fixing this problem.


'CWE-307'

In [20]:
## https://cwe.mitre.org/data/definitions/307.html

In [21]:
## Classifier hyperparameter tuning

In [22]:
from sklearn.model_selection import GridSearchCV
classifier = LinearSVC()#class_weight='balanced'
pipe = Pipeline([('vect', vectorizer), ('cls', classifier)])
#hyperparameters
parameters = {'cls__loss': ('hinge', 'squared_hinge'),#default: squared_hinge
              'cls__dual': (True, False),#default: True
              'cls__multi_class': ('ovr', 'crammer_singer'),#default: ovr
              'cls__max_iter': (1000, 2000),#default: 1000
             }
gs = GridSearchCV(pipe, parameters, scoring='f1_micro', cv=10, error_score=np.nan)

gs = gs.fit(cwe_texts['text'], cwe_texts['label'])
print(gs.best_score_)
print(gs.best_params_)

C:\Users\ossi.jormakka\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\ossi.jormakka\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:559: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Unsupported set of arguments: The combination of penalty='l2' and loss='hinge' are not supported when dual=False, Parameters: penalty='l2', loss='hinge', dual=False

  FitFailedWarning)
C:\Users\ossi.jormakka\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  Dep

0.680466472303207
{'cls__dual': True, 'cls__loss': 'squared_hinge', 'cls__max_iter': 1000, 'cls__multi_class': 'crammer_singer'}


In [38]:
## Find optimal dataset size to fit the classifier

In [70]:
sample_sizes = [100, 200, 500, 1000, 2000, 3000, 4000, 5000, 6000, 8000, 10000, 12000, 14000]

data = shuffle(nvd_vulns)
print('data: ' + str(len(data)))

results = []
for size in sample_sizes:
    samples = sample_cwe(data, size)
    score = cross_val_score(pipe, samples['text'], samples['label'], cv=10, scoring="f1_micro")
    score = np.mean(score)
    results.append(score)
    print(str(size) + ': ' + str(score))

data: 18755
100: 0.7225778783085111
200: 0.7460082925565177
500: 0.7879766200427929
1000: 0.7963461284982685
2000: 0.8086168524166265
3000: 0.8096809318170927
4000: 0.8134996216226391
5000: 0.8108861222441972
6000: 0.8131727804448643
8000: 0.8123744513611921
10000: 0.8129660682205431
12000: 0.8099118221644698
14000: 0.8125795139157079
